In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm

In [7]:
#load data
sale_data = pd.read_csv("..//data//train.csv")

/var/folders/1k/b5ntvdmn14d6y71h586c2n540000gn/T/ipykernel_13370/1554227962.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sale_data = pd.read_csv("..//data//train.csv")


In [6]:
#index the date
initial_date = datetime.strptime('2013-01-01', '%Y-%m-%d')
sale_data['date_index'] = 0
for i in tqdm(sale_data.index):
    temp_date = datetime.strptime(sale_data.loc[i,'Date'], '%Y-%m-%d')
    days_gap = (temp_date - initial_date).days 
    sale_data.loc[i,"date_index"] = days_gap

100%|██████████| 1017209/1017209 [00:35<00:00, 28895.35it/s]


In [8]:
# delete the data of 2013-12 and weekends
index_12 = set(list(sale_data[sale_data['Date'].apply(lambda x: x[5:7] == '12')]['date_index']) + list(sale_data[sale_data.DayOfWeek == 6]['date_index']) + list(sale_data[sale_data.DayOfWeek == 7]['date_index']))
sale_data = sale_data.drop(index = sale_data[sale_data['Date'].apply(lambda x: x[5:7] == '12')].index)
sale_data = sale_data.drop(index = sale_data[sale_data.DayOfWeek == 6].index)
sale_data = sale_data.drop(index = sale_data[sale_data.DayOfWeek == 7].index)
sale_data['DayOfWeek'].unique()

array([5, 4, 3, 2, 1])

In [8]:
#delete the columns of Date, Customers, StateHoliday
sale_data = sale_data.drop(columns = ['Date','Customers','StateHoliday'])
sale_data.reset_index(inplace = True,drop= True)
sale_data

,Store,DayOfWeek,Sales,Open,Promo,SchoolHoliday,date_index
0,1,5,5263,1,1,1,941
1,2,5,6064,1,1,1,941
2,3,5,8314,1,1,1,941
3,4,5,13995,1,1,1,941
4,5,5,4822,1,1,1,941
...,...,...,...,...,...,...,...
681709,1111,2,0,0,0,1,0
681710,1112,2,0,0,0,1,0
681711,1113,2,0,0,0,1,0
681712,1114,2,0,0,0,1,0


In [9]:
#one-hot encoding for DayOfWeek
sale_data_onehot = pd.get_dummies(sale_data,columns=['DayOfWeek'],dtype=float)
sale_data_onehot = sale_data_onehot.drop(columns = ['DayOfWeek_1'])

,Store,Sales,Open,Promo,SchoolHoliday,date_index,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5
0,1,5263,1,1,1,941,0.0,0.0,0.0,1.0
1,2,6064,1,1,1,941,0.0,0.0,0.0,1.0
2,3,8314,1,1,1,941,0.0,0.0,0.0,1.0
3,4,13995,1,1,1,941,0.0,0.0,0.0,1.0
4,5,4822,1,1,1,941,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
681709,1111,0,0,0,1,0,1.0,0.0,0.0,0.0
681710,1112,0,0,0,1,0,1.0,0.0,0.0,0.0
681711,1113,0,0,0,1,0,1.0,0.0,0.0,0.0
681712,1114,0,0,0,1,0,1.0,0.0,0.0,0.0


In [10]:
#generate the feature vector and labels
groups_store = sale_data_onehot.groupby('Store')
labels = np.zeros((1115,942))
column_feature = list(sale_data_onehot.columns)
column_feature.remove('Sales')
column_feature.remove('Store')
column_feature.remove('Open')
feature_vector = np.zeros((1115,942,len(column_feature)))
a = 0
for group_name, df in tqdm(enumerate(groups_store),total=len(groups_store), desc='Processing groups'):
    df = df[1]
    for j in df.index:
        if df.loc[j,'Open'] == 1 and df.loc[j,'Sales'] != 0:
            index_date = df.loc[j,'date_index']
            labels[group_name,index_date] = df.loc[j,'Sales']
            feature_vector[group_name,index_date,:] = df.loc[j,column_feature]

feature_final = []
label_final = []
j_list = []
for k in range(feature_vector.shape[0]):
    feature_k = []
    label_k = []
    for i in range(942):
        if i not in index_12:
            feature_k.append(feature_vector[k,i,:])
            label_k.append(labels[k,i])
    j = np.argwhere(np.array(label_k) == 0).shape[0]/len(label_k)
    j_list.append(j)
    #if j< 0.05:
    feature_final.append(feature_k)
    label_final.append(label_k)
feature_final = np.array(feature_final)
label_final = np.array(label_final)
feature_final.shape,label_final.shape

Processing groups: 100%|██████████| 1115/1115 [01:15<00:00, 14.71it/s]


In [12]:
#save the data in the numpy files
np.save("..//np_files//label.npy",label_final)
np.save("..//np_files//feature.npy",feature_final)